# Train & 5-Fold Evaluation for TCN + DBN

Simply run all blocks in this Jupyter Notebook to execute 5-fold cross-validation training and evaluation for our TCN-based tempo, beat, and downbeat estimator. You'll find final performance metrics in the last code block.

**Setup Reminder:**
1. Ensure you've properly set up the `mirdata` module as described in **README.md**.
2. Modify **Step 2**: Update the path to your `mazurkaBL.h5` dataset (preprocessed using `data_preprocess.ipynb`).
3. Modify **Step 3**: Adjust your 5-fold split (`csv_dir` and `csv_filename`) to match your directory structure.

**Acknowledgements:**
This notebook builds upon and extends previous open-source work & TCN paper.
- **S. Böck & M. E. P. Davies**, *Tempo, Beat, and Downbeat Tutorial* (2020). [eBook](https://tempobeatdownbeat.github.io/tutorial/intro.html)
- **S. Böck & M. E. P. Davies (2020)**, *Deconstruct, Analyse, Reconstruct: How to Improve Tempo, Beat, and Downbeat Estimation*, *Proc. ISMIR*, pp. 574–582. [Paper](https://program.ismir2020.net/static/final_papers/223.pdf)

**Modification:** 
1. We remove the tempo estimation branch from the TCN, since the MazuraBL dataset not contain tempo annotation.
2. We modify the params of DBN Downbeat Tracker, since all MazurkaBL music are 3 beats per bar, each bar one downbeat.

# 5-Fold Cross-validation Results of TCN on MazurkaBL Dataset
**Step 5 Results Summary (step-by-step run this code notebook to get these results)**
| Fold | Beat F1 (FINAL) | Beat F1 (BEST - reported) | Downbeat F1 (FINAL) | Downbeat F1 (BEST - reported) |
|------|----------------|-----------------|--------------------|---------------------|
| 0    | 0.6286         | 0.6287          | 0.3072             | 0.3090              |
| 1    | 0.6113         | 0.6108          | 0.3053             | 0.3074              |
| 2    | 0.5771         | 0.5792          | 0.2817             | 0.2814              |
| 3    | 0.6199         | 0.6250          | 0.3172             | 0.3206              |
| 4    | 0.6026         | 0.6031          | 0.2989             | 0.2992              |
| **Avg ± Std** | 0.6079 ± 0.0177 | **0.6094 ± 0.0177** | 0.3020 ± 0.0118 | **0.3035 ± 0.0130** |

**(Step 6 We verified our reproduced TCN has similar performance on GTZAN dataset as Böck et al., refer to their [Colab_Notebook](https://colab.research.google.com/drive/1tuOqNyO9gdMmYJsj33fP_QOfpRsm2tmt?usp=sharing))**


## 1. Setup Environment (show package-versions)


In [31]:
import os, sys, platform, importlib, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import madmom
import librosa
from tqdm import tqdm
from scipy.ndimage import maximum_filter1d
from scipy.interpolate import interp1d
from scipy.signal import argrelmax
from pathlib import Path
from glob import glob
from keras.utils import Sequence
from keras.callbacks import CSVLogger

# Only print ERROR, ignore warnings
import warnings
warnings.filterwarnings('ignore')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  

def where(mod):
    try:
        m = importlib.import_module(mod)
        return getattr(m, "__version__", "unknown"), getattr(m, "__file__", "n/a")
    except Exception as e:
        return f"IMPORT FAILED: {type(e).__name__}: {e}", "n/a"

print("Python:", sys.version.split()[0], "|", platform.platform())
print("Interpreter:", sys.executable)

for mod in ["numpy", "tensorflow", "keras", "tensorflow_addons",
            "librosa", "madmom", "mir_eval", "mirdata"]:
    ver, path = where(mod)
    print(f"{mod:20s} -> {ver:25s} | {path}")

# Limit the GPU occupancy of Tensorflow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Python: 3.9.23 | Linux-6.8.0-60-generic-x86_64-with-glibc2.35
Interpreter: /media/datadisk/home/22828187/conda_env/miniconda3/envs/beat_mir/bin/python
numpy                -> 1.23.5                    | /media/datadisk/home/22828187/conda_env/miniconda3/envs/beat_mir/lib/python3.9/site-packages/numpy/__init__.py
tensorflow           -> 2.15.1                    | /media/datadisk/home/22828187/conda_env/miniconda3/envs/beat_mir/lib/python3.9/site-packages/tensorflow/__init__.py
keras                -> 2.15.0                    | /media/datadisk/home/22828187/conda_env/miniconda3/envs/beat_mir/lib/python3.9/site-packages/keras/__init__.py
tensorflow_addons    -> 0.23.0                    | /media/datadisk/home/22828187/conda_env/miniconda3/envs/beat_mir/lib/python3.9/site-packages/tensorflow_addons/__init__.py
librosa              -> 0.10.2                    | /media/datadisk/home/22828187/conda_env/miniconda3/envs/beat_mir/lib/python3.9/site-packages/librosa/__init__.py
madmom         

## 2. Load MazurkaBL (packed h5 files) with mirdata
- `TO BE Modify: make the pre-processed mazurkaBL.h5 data_home path match yours.`
- data_home="/media/datadisk/home/22828187/zhanh/202505_dynest_data/workspaces/hdf5s/mazurka_sr22050"

In [32]:
from mirdata.datasets import mazurka_h5
mazurka = mazurka_h5.Dataset(version="local", data_home="/media/datadisk/home/22828187/zhanh/202505_dynest_data/workspaces/hdf5s/mazurka_sr22050")
tracks = mazurka.load_tracks() # for training
print("num tracks:", len(mazurka.track_ids))
print("sample ids:", mazurka.track_ids[:5])

num tracks: 1999
sample ids: ['mazurka06-1/pid1263-01', 'mazurka06-1/pid52932-01', 'mazurka06-1/pid9048-01', 'mazurka06-1/pid9050-01', 'mazurka06-1/pid9054-01']


## 3. Load the 5-fold splits via CSVs

We follow a 5-fold cross validation protocol, same as other benchmark in our paper. All 1,999 Mazurka tracks are partitioned into 5 folds, each with its own CSV file. or each fold, one subset is used for testing, while the remaining four folds are used for training (excluded validation set). 
 - `TO BE Modify: make the 5-fold split csv_dir & csv_filename match yours.`
 - csv_dir = "/media/datadisk/home/22828187/zhanh/202505_dynest_data/workspaces"
 - csv_files = sorted(glob(f"{csv_dir}/split_5fold_fold*_seed86.csv"))

In [33]:
def load_split(csv_path, mazurka):
    """Load a 5fold CSV, return train/valid/test list."""
    df = pd.read_csv(csv_path)
    df.columns = [c.strip().lower() for c in df.columns]

    def to_track_id(row):
        stem = Path(str(row["h5_name"])).stem
        opus = str(row["opus"])
        return f"{opus}/{stem}"

    df["track_id"] = df.apply(to_track_id, axis=1)

    train_ids = df.loc[df["split"].str.lower() == "train", "track_id"].tolist()
    valid_ids = df.loc[df["split"].str.lower().isin(["valid","val"]), "track_id"].tolist()
    test_ids  = df.loc[df["split"].str.lower() == "test",  "track_id"].tolist()

    all_ids = set(mazurka.track_ids)
    for name, ids in {"train":train_ids, "valid":valid_ids, "test":test_ids}.items():
        missing = [i for i in ids if i not in all_ids]
        if missing:
            print(f"[WARN] Broken pro-precess h5 dataset, {csv_path} {name} has {len(missing)} ID not included.")
    return train_ids, valid_ids, test_ids

In [34]:
csv_dir = "/media/datadisk/home/22828187/zhanh/202505_dynest_data/workspaces"
csv_files = sorted(glob(f"{csv_dir}/split_5fold_fold*_seed86.csv"))

fold_splits = {}
for f in csv_files:
    fold = int(Path(f).stem.split("_fold")[1].split("_")[0])
    train_ids, valid_ids, test_ids = load_split(f, mazurka)
    fold_splits[fold] = {"train": train_ids, "valid": valid_ids, "test":  test_ids}
    print(f"Fold {fold}: train={len(train_ids)}, valid={len(valid_ids)}, test={len(test_ids)}")

Fold 0: train=1184, valid=407, test=397
Fold 1: train=1207, valid=374, test=407
Fold 2: train=1208, valid=406, test=374
Fold 3: train=1178, valid=404, test=406
Fold 4: train=1187, valid=397, test=404


# 4. Define TCN Network, Audio Processor, and Model Prediction & Evalaution Functions

We create a multi-task model to jointly predict tempo, beats and doenbeats, which mostly follows our ISMIR 2020 paper "Deconstruct, analyse, reconstruct: how to improve tempo, beat, and downbeat estimation.".

The heart of the network is a TCN (temporal convolutional network) with 11 TCN layers with increasing dilation rates.

In [35]:
#@title Define the TCN model
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Activation, Conv1D, Conv2D, MaxPooling2D, Reshape, Dropout, SpatialDropout1D, GaussianNoise, GlobalAveragePooling1D, Concatenate, Add

def residual_block(x, i, activation, num_filters, kernel_size, padding, dropout_rate=0, name=''):
    name = f"{name}_dilation_{i}"
    res_x = Conv1D(num_filters, 1, padding='same', name=name + '_1x1_conv_residual')(x)

    conv_1 = Conv1D(num_filters, kernel_size, dilation_rate=i,   padding=padding, name=name + '_dilated_conv_1')(x)
    conv_2 = Conv1D(num_filters, kernel_size, dilation_rate=2*i, padding=padding, name=name + '_dilated_conv_2')(x)

    concat = Concatenate(name=name + '_concat')([conv_1, conv_2])
    x = Activation(activation, name=name + '_activation')(concat) 
    x = SpatialDropout1D(dropout_rate, name=f"{name}_spatial_dropout_{dropout_rate}")(x)
    x = Conv1D(num_filters, 1, padding='same', name=name + '_1x1_conv')(x)

    return Add(name=name + '_merge_residual')([res_x, x]), x

class TCN:
    def __init__(self, num_filters=20, kernel_size=5,
                 dilations=[1,2,4,8,16,32,64,128,256,512,1024],
                 activation='elu', padding='same', dropout_rate=0.15, name='tcn'):
        if padding not in ('causal', 'same'):
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.dilations = dilations
        self.activation = activation
        self.padding = padding
        self.dropout_rate = dropout_rate
        self.name = name

    def __call__(self, inputs):
        x = inputs
        skip_connections = []
        for i, nf in zip(self.dilations, self.num_filters):
            x, skip_out = residual_block(
                x, i, self.activation, nf, self.kernel_size, self.padding, self.dropout_rate, name=self.name
            )
            skip_connections.append(skip_out)
        x = Activation(self.activation, name=self.name + '_activation')(x)
        skip = Add(name=self.name + '_merge_skip_connections')(skip_connections)
        return x, skip

def create_model(input_shape, num_filters=20, num_dilations=11, kernel_size=5, activation='elu', dropout_rate=0.15):
    inp = Input(shape=input_shape)

    conv_1 = Conv2D(num_filters, (3, 3), padding='valid', name='conv_1_conv')(inp)
    conv_1 = Activation(activation, name='conv_1_activation')(conv_1)
    conv_1 = MaxPooling2D((1, 3), name='conv_1_max_pooling')(conv_1)
    conv_1 = Dropout(dropout_rate, name='conv_1_dropout')(conv_1)

    conv_2 = Conv2D(num_filters, (1, 10), padding='valid', name='conv_2_conv')(conv_1)
    conv_2 = Activation(activation, name='conv_2_activation')(conv_2)
    conv_2 = MaxPooling2D((1, 3), name='conv_2_max_pooling')(conv_2)
    conv_2 = Dropout(dropout_rate, name='conv_2_dropout')(conv_2)

    conv_3 = Conv2D(num_filters, (3, 3), padding='valid', name='conv_3_conv')(conv_2)
    conv_3 = Activation(activation, name='conv_3_activation')(conv_3)
    conv_3 = MaxPooling2D((1, 3), name='conv_3_max_pooling')(conv_3)
    conv_3 = Dropout(dropout_rate, name='conv_3_dropout')(conv_3)

    x = Reshape((-1, num_filters), name='tcn_input_reshape')(conv_3)

    dilations = [2 ** i for i in range(num_dilations)]
    tcn, skip = TCN(
        num_filters=[num_filters] * len(dilations),
        kernel_size=kernel_size,
        dilations=dilations,
        activation=activation,
        padding='same',
        dropout_rate=dropout_rate,
    )(x)

    beats = Dropout(dropout_rate, name='beats_dropout')(tcn)
    beats = Dense(1, name='beats_dense')(beats)
    beats = Activation('sigmoid', name='beats')(beats)

    downbeats = Dropout(dropout_rate, name='downbeats_dropout')(tcn)
    downbeats = Dense(1, name='downbeats_dense')(downbeats)
    downbeats = Activation('sigmoid', name='downbeats')(downbeats)

    # Remove tempo head as MazurkaBL dataset not contain tempo annotation
    # tempo = Dropout(dropout_rate, name='tempo_dropout')(skip)
    # tempo = GlobalAveragePooling1D(name='tempo_global_average_pooling')(tempo)
    # tempo = GaussianNoise(dropout_rate, name='tempo_noise')(tempo)
    # tempo = Dense(300, name='tempo_dense')(tempo)
    # tempo = Activation('softmax', name='tempo')(tempo)
    # return Model(inp, outputs=[beats, downbeats, tempo])
    
    return Model(inp, outputs=[beats, downbeats])

In [36]:
#@title PreProcessor - STFT Transform, cnn_pad and infer_tempo
from madmom.processors import ParallelProcessor, SequentialProcessor
from madmom.audio.signal import SignalProcessor, FramedSignalProcessor
from madmom.audio.stft import ShortTimeFourierTransformProcessor
from madmom.audio.spectrogram import FilteredSpectrogramProcessor, LogarithmicSpectrogramProcessor

# Use same audio pre-processing FFT params of our MazurkaBL.h5 files
FPS = 50
FFT_SIZE = 1024
SAMPLE_RATE = 22050

# Default Params of TCN on GTZAN (produce similar results as above params, but much more GPU Mem taken)
# FPS = 100
# FFT_SIZE = 2048
# SAMPLE_RATE = 44100

NUM_BANDS = 12
MASK_VALUE = -1

# define pre-processor
class PreProcessor(SequentialProcessor):
    def __init__(self, frame_size=FFT_SIZE, num_bands=NUM_BANDS, log=np.log, add=1e-6, fps=FPS):
        sig = SignalProcessor(num_channels=1, sample_rate=SAMPLE_RATE)
        frames = FramedSignalProcessor(frame_size=frame_size, fps=fps)
        stft = ShortTimeFourierTransformProcessor()
        filt = FilteredSpectrogramProcessor(num_bands=num_bands)
        spec = LogarithmicSpectrogramProcessor(log=log, add=add)
        super(PreProcessor, self).__init__((sig, frames, stft, filt, spec, np.array))
        self.fps = fps


def cnn_pad(data, pad_frames):
    """Pad the data by repeating the first and last frame N times."""
    pad_start = np.repeat(data[:1], pad_frames, axis=0)
    pad_stop = np.repeat(data[-1:], pad_frames, axis=0)
    return np.concatenate((pad_start, data, pad_stop))


# def infer_tempo(beats, key=None, hist_smooth=15, fps=FPS, no_tempo=MASK_VALUE):
#     ibis = np.diff(beats) * fps
#     ibis_rounded = np.round(ibis).astype(int)
#     bins = np.bincount(ibis_rounded)
#     if not bins.any():
#         return no_tempo
#     if hist_smooth > 0:
#         bins = madmom.audio.signal.smooth(bins, hist_smooth)
#     intervals = np.arange(len(bins)) 
#     tempi = 60.0 * fps / intervals
#     tempi[0] = 0 
#     peaks = argrelmax(bins, mode='wrap')[0]
#     if len(peaks) == 0:
#         return no_tempo
#     best = peaks[np.argmax(bins[peaks])]
#     tempo_val = tempi[best]
#     if not np.isfinite(tempo_val) or tempo_val <= 0:
#         return no_tempo
#     return tempo_val

In [37]:
#@title DataSequence (new) - with widen functions restored
# MazurkaBL dataset has no tempo annotation, we comment out this attribute
class DataSequence(Sequence):
    def __init__(self, tracks, pre_processor, num_tempo_bins=300, pad_frames=None,
                 win_s=30.0, hop_s=30.0):
        self.fps = pre_processor.fps
        self.win = int(round(win_s * self.fps))
        self.hop = int(round(hop_s * self.fps))
        self.pad = pad_frames
        self.X, self.Yb, self.Yd, self.Yt, self.segs = {}, {}, {}, {}, []
        for key, t in tracks.items():
            y, sr = t.audio
            X = pre_processor(madmom.audio.Signal(y, sr)).astype('float32')
            T = len(X)
            bs = t.beats.times
            beat = madmom.utils.quantize_events(bs, fps=self.fps, length=T).astype('float32')
            dbs = bs[t.beats.positions.astype(int) == 1]
            downbeat = madmom.utils.quantize_events(dbs, fps=self.fps, length=T).astype('float32')
            # tempo = int(round(infer_tempo(bs, key, fps=self.fps)))
            # while tempo >= num_tempo_bins: tempo //= 2
            # tempo = keras.utils.to_categorical(tempo, num_tempo_bins, dtype='float32')
            # self.X[key], self.Yb[key], self.Yd[key], self.Yt[key] = X, beat, downbeat, tempo
            self.X[key], self.Yb[key], self.Yd[key] = X, beat, downbeat
            for s in range(0, T - self.win + 1, self.hop):
                self.segs.append((key, s, s + self.win))
        self.N = len(self.segs)

    def __len__(self): return self.N

    def __getitem__(self, i):
        key, a, b = self.segs[i]
        x = self.X[key][a:b]
        if self.pad: x = cnn_pad(x, self.pad)
        y = {
            'beats':     self.Yb[key][a:b][None, ..., None],
            'downbeats': self.Yd[key][a:b][None, ..., None],
            # 'tempo':     self.Yt[key][None, ...]
        }
        return x[None, ..., None], y

    # --- widen targets (Didn't applied this data augmentation in our work) ---
    def widen_beat_targets(self, size=3, value=0.5):
        for y in self.Yb.values():
            if np.allclose(y, MASK_VALUE): continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def widen_downbeat_targets(self, size=3, value=0.5):
        for y in self.Yd.values():
            if np.allclose(y, MASK_VALUE): continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    # def widen_tempo_targets(self, size=3, value=0.5):
    #     for y in self.Yt.values():
    #         if np.allclose(y, MASK_VALUE): continue
    #         np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def append(self, other):
        assert not any(k in self.X for k in other.X), 'IDs must be unique'
        self.X.update(other.X)
        self.Yb.update(other.Yb)
        self.Yd.update(other.Yd)
        # self.Yt.update(other.Yt)
        self.segs.extend(other.segs)
        self.N = len(self.segs)

In [38]:
#@title Masked Loss code based on: https://github.com/keras-team/keras/issues/3893
def build_masked_loss(loss_function, mask_value=MASK_VALUE):
    """Builds a loss function that masks based on targets

    Args:
        loss_function: The loss function to mask
        mask_value: The value to mask in the targets

    Returns:
        function: a loss function that acts like loss_function with masked inputs
    """

    def masked_loss_function(y_true, y_pred):
        mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        return loss_function(y_true * mask, y_pred * mask)

    return masked_loss_function


def masked_accuracy(y_true, y_pred):
    total = K.sum(K.not_equal(y_true, MASK_VALUE))
    correct = K.sum(K.equal(y_true, K.round(y_pred)))
    return correct / total

In [39]:
#@title Func for Model Prediction & Evaluation  (tempo head removed)
def predict_beats_downbeats(model, dataset, fps=FPS, dedup_frames=2, desc="Predicting"):
    fps = getattr(dataset, "fps", getattr(getattr(dataset, "pp", None), "fps", fps))
    dedup_sec = dedup_frames / float(fps)
    # Performance: min_bpm 55 < 75 < 90
    # Performance: max_bpm 240 = 215 > 200 > 180
    
    def _beat_tracker(beats_act):
        proc = madmom.features.beats.DBNBeatTrackingProcessor(
            min_bpm=90.0, max_bpm=215.0, fps=fps, transition_lambda=100, threshold=0.05)
        return proc(beats_act)

    def _downbeat_tracker(beats_act, downbeats_act):
        combined = np.vstack([np.maximum(beats_act - downbeats_act, 0.0), downbeats_act]).T
        proc = madmom.features.downbeats.DBNDownBeatTrackingProcessor(
            beats_per_bar=[3], min_bpm=90.0, max_bpm=215.0, fps=fps, transition_lambda=100)
        out = proc(combined)
        return out[:, 0] if len(out) else np.empty((0,), dtype=float)

    def _merge_segments(times_list):
        if not times_list: return np.empty((0,), dtype=float)
        t = np.sort(np.concatenate(times_list))
        if len(t) == 0: return t
        keep = [t[0]]
        for x in t[1:]:
            if x - keep[-1] >= dedup_sec:
                keep.append(x)
        return np.asarray(keep)

    beats_buf, down_buf = {}, {}

    for i in tqdm(range(len(dataset)), desc=desc):
        # If this song needs to be separated into 30s segments
        if hasattr(dataset, "segs"):
            k, a, b = dataset.segs[i]          # a,b are frame indices
            offset_sec = a / float(fps)        # Start-time offset
        else:
            # This song is already in 30s segments format
            k = dataset.ids[i]
            offset_sec = 0.0

        x, _ = dataset[i]                      # x: (1, T, F, 1); labels ignored
        # === tempo head removed: expect only two outputs ===
        b_act, d_act = model.predict(x, verbose=0)   # (1, T, 1) each
        b_act = b_act.squeeze()
        d_act = d_act.squeeze()

        # ---- Add the start-time offset into the detected results ----
        bt = _beat_tracker(b_act) + offset_sec
        dbt = _downbeat_tracker(b_act, d_act) + offset_sec

        beats_buf.setdefault(k, []).append(bt)
        down_buf.setdefault(k, []).append(dbt)

    detections = {
        k: {
            "beats": _merge_segments(beats_buf[k]),
            "downbeats": _merge_segments(down_buf.get(k, [])),
        }
        for k in beats_buf
    }
    return detections


def evaluate_beats_and_downbeats(detections, beat_ann, downbeat_ann):
    """
    detections:   dict[key] -> {'beats': 1D np.ndarray(sec), 'downbeats': 1D np.ndarray(sec)}
    beat_ann:     dict[key] -> 1D np.ndarray(sec) of reference beat times
    downbeat_ann: dict[key] -> 1D np.ndarray(sec) of reference downbeat times
    Returns: {"beat": BeatMeanEvaluation or None, "downbeat": BeatMeanEvaluation or None}
    """
    beat_evals, down_evals = [], []
    for k, det in detections.items():
        if k in beat_ann:
            beat_evals.append(madmom.evaluation.beats.BeatEvaluation(det['beats'], beat_ann[k]))
        if k in downbeat_ann:
            down_evals.append(madmom.evaluation.beats.BeatEvaluation(det['downbeats'], downbeat_ann[k], downbeats=True))
    beat_mean     = madmom.evaluation.beats.BeatMeanEvaluation(beat_evals) if beat_evals else None
    downbeat_mean = madmom.evaluation.beats.BeatMeanEvaluation(down_evals) if down_evals else None
    if beat_mean:
        print("\nBeat evaluation"); print(beat_mean)
    if downbeat_mean:
        print("\nDownbeat evaluation"); print(downbeat_mean)
    return {"beat": beat_mean, "downbeat": downbeat_mean}


# 5. Run 5-Fold Train & Evaluation for TCN

This notebook trains a TCN model (beat / downbeat / tempo heads) with **5-fold** splits.  
For each fold, we train up to 100 epochs with validation-based early stopping, save the best checkpoint, then evaluate on the test split and finally report the **mean ± std** across folds.

### Before running: ensure *Paths** and **Training Hyper-Params (optional)**
- `PreProcessor` sample rate / FPS if needed.
- `mazurka_h5` data path (your local H5 root).
- 5-fold CSV path(s) for `train/valid/test`.
- `epochs` (default 100), optimizer / LR, and callbacks.

### Workflow of code below
1. Build `train / valid / test` from the selected fold.
2. Create `DataSequence` (segments = 30 s, no augmentation).
3. `model.fit(...)` with `ModelCheckpoint` (best **val loss**), `ReduceLROnPlateau`, `EarlyStopping`, `TensorBoard`.
4. Load `model_best.h5`, run inference, track beats & downbeats via madmom DBN, and evaluate.
5. Repeat for all 5 folds and print averaged F1 (beat & downbeat).

> Tip: keep training and evaluation in **separate cells** to keep logs clean.  
> Output checkpoints are saved under `checkpoint/fold{i}_eps{epochs}/`.

In [ ]:
# ========= 5-FOLD TRAINING (save BEST & FINAL ckpt, not use data augmentation) =========
epochs, pad_frames = 120, 2
ckpt_root = Path.cwd() / "checkpoint"

for fold in range(5):
      print(f"\n=== TRAIN fold {fold} ===")
      train_ids = fold_splits[fold]["train"]
      valid_ids = fold_splits[fold]["valid"]

      pp = PreProcessor()
      train = DataSequence({k: tracks[k] for k in train_ids}, pre_processor=pp, pad_frames=pad_frames)
      valid = DataSequence({k: tracks[k] for k in valid_ids}, pre_processor=pp, pad_frames=pad_frames)

      input_shape = (None,) + train[0][0].shape[-2:]
      model = create_model(input_shape)
      model.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss=[build_masked_loss(K.binary_crossentropy)]*2,   # beat + downbeat loss
                  # loss=[build_masked_loss(K.binary_crossentropy)]*3, # tempo loss removed
                  metrics=['binary_accuracy'])
      outdir = ckpt_root / f"fold{fold}_eps{epochs}"
      outdir.mkdir(parents=True, exist_ok=True)

      cb = [keras.callbacks.ModelCheckpoint(str(outdir/"model_best.h5"),save_best_only=True, monitor="val_loss"),
            keras.callbacks.ReduceLROnPlateau(patience=10, factor=0.2, min_lr=1e-7, monitor="val_loss", verbose=1),
            keras.callbacks.EarlyStopping(patience=80, monitor="val_loss"),
            CSVLogger(str(outdir/"train_log.txt"), append=True, separator="\t")]
      model.fit(train, validation_data=valid, epochs=epochs, callbacks=cb, shuffle=True, verbose=1)
      model.save(outdir/"model_final.h5")

In [42]:
# ========= 5-FOLD Evaluation (report BEST & FINAL) =========
beat_ann     = {k: v.beats.times for k, v in tracks.items() if v.beats is not None}
downbeat_ann = {k: v.beats.times[v.beats.positions.astype(int)==1] for k, v in tracks.items() if v.beats is not None}

epochs, pad_frames = 120, 2
ckpt_root = Path.cwd() / "checkpoint"

summary_best_beats, summary_best_down = [], []
summary_final_beats, summary_final_down = [], []

for fold in range(5):
    print(f"\n--- EVAL fold {fold} ---")
    pp = PreProcessor()
    test_ids = fold_splits[fold]["test"]
    test = DataSequence({k: tracks[k] for k in test_ids}, pre_processor=pp, pad_frames=pad_frames)

    input_shape = (None,) + test[0][0].shape[-2:]
    model = create_model(input_shape)
    model.compile(optimizer="adam",
                    loss=[build_masked_loss(K.binary_crossentropy)]*2,
                    metrics=['binary_accuracy'])
                    # loss=[build_masked_loss(K.binary_crossentropy)]*3, # tempo head removed

    outdir = ckpt_root / f"fold{fold}_eps{epochs}"

    # --- BEST ---
    model.load_weights(str(outdir / "model_best.h5"))
    detections = predict_beats_downbeats(model, test, fps=pp.fps)
    scores = evaluate_beats_and_downbeats(detections, beat_ann, downbeat_ann)
    b_f1_best, d_f1_best = float(scores["beat"].fmeasure), float(scores["downbeat"].fmeasure)
    print(f"Fold {fold}  [BEST ] Beat F1: {b_f1_best:.4f} | Downbeat F1: {d_f1_best:.4f}")
    summary_best_beats.append(b_f1_best); summary_best_down.append(d_f1_best)

    # --- FINAL ---
    model.load_weights(str(outdir / "model_final.h5"))
    detections = predict_beats_downbeats(model, test, fps=pp.fps)
    scores = evaluate_beats_and_downbeats(detections, beat_ann, downbeat_ann)
    b_f1_final, d_f1_final = float(scores["beat"].fmeasure), float(scores["downbeat"].fmeasure)
    print(f"Fold {fold}  [FINAL] Beat F1: {b_f1_final:.4f} | Downbeat F1: {d_f1_final:.4f}")
    summary_final_beats.append(b_f1_final); summary_final_down.append(d_f1_final)


--- EVAL fold 0 ---


Predicting: 100%|██████████| 1992/1992 [01:48<00:00, 18.42it/s]



Beat evaluation
mean for 397 files
  F-measure: 0.629 P-score: 0.635 Cemgil: 0.502 Goto: 0.010 CMLc: 0.043 CMLt: 0.403 AMLc: 0.051 AMLt: 0.422 D: 0.564 Dg: 0.393

Downbeat evaluation
mean for 397 files
  F-measure: 0.307 P-score: 0.334 Cemgil: 0.246 Goto: 0.000 CMLc: 0.001 CMLt: 0.001 AMLc: 0.088 AMLt: 0.467 D: 0.552 Dg: 0.320
Fold 0  [BEST ] Beat F1: 0.6286 | Downbeat F1: 0.3072


Predicting: 100%|██████████| 1992/1992 [01:49<00:00, 18.25it/s]



Beat evaluation
mean for 397 files
  F-measure: 0.629 P-score: 0.636 Cemgil: 0.501 Goto: 0.010 CMLc: 0.043 CMLt: 0.404 AMLc: 0.051 AMLt: 0.422 D: 0.566 Dg: 0.392

Downbeat evaluation
mean for 397 files
  F-measure: 0.309 P-score: 0.334 Cemgil: 0.246 Goto: 0.000 CMLc: 0.001 CMLt: 0.001 AMLc: 0.090 AMLt: 0.468 D: 0.558 Dg: 0.323
Fold 0  [FINAL] Beat F1: 0.6287 | Downbeat F1: 0.3090

--- EVAL fold 1 ---


...


Beat evaluation
mean for 406 files
  F-measure: 0.625 P-score: 0.596 Cemgil: 0.507 Goto: 0.002 CMLc: 0.042 CMLt: 0.362 AMLc: 0.063 AMLt: 0.422 D: 0.585 Dg: 0.392

Downbeat evaluation
mean for 406 files
  F-measure: 0.321 P-score: 0.333 Cemgil: 0.260 Goto: 0.000 CMLc: 0.002 CMLt: 0.003 AMLc: 0.073 AMLt: 0.409 D: 0.577 Dg: 0.340
Fold 3  [FINAL] Beat F1: 0.6250 | Downbeat F1: 0.3206

--- EVAL fold 4 ---


Predicting: 100%|██████████| 1746/1746 [01:58<00:00, 14.68it/s]



Beat evaluation
mean for 404 files
  F-measure: 0.603 P-score: 0.613 Cemgil: 0.474 Goto: 0.007 CMLc: 0.041 CMLt: 0.367 AMLc: 0.050 AMLt: 0.387 D: 0.543 Dg: 0.328

Downbeat evaluation
mean for 404 files
  F-measure: 0.299 P-score: 0.337 Cemgil: 0.237 Goto: 0.000 CMLc: 0.001 CMLt: 0.001 AMLc: 0.076 AMLt: 0.416 D: 0.513 Dg: 0.275
Fold 4  [BEST ] Beat F1: 0.6026 | Downbeat F1: 0.2989


Predicting: 100%|██████████| 1746/1746 [02:13<00:00, 13.12it/s]



Beat evaluation
mean for 404 files
  F-measure: 0.603 P-score: 0.613 Cemgil: 0.475 Goto: 0.007 CMLc: 0.041 CMLt: 0.367 AMLc: 0.051 AMLt: 0.388 D: 0.547 Dg: 0.328

Downbeat evaluation
mean for 404 files
  F-measure: 0.299 P-score: 0.336 Cemgil: 0.237 Goto: 0.000 CMLc: 0.001 CMLt: 0.001 AMLc: 0.076 AMLt: 0.420 D: 0.521 Dg: 0.284
Fold 4  [FINAL] Beat F1: 0.6031 | Downbeat F1: 0.2992


In [ ]:
# ========= 5-FOLD Summary =========
print("\n=== 5-Fold Summary (F-measure) (90,215) ===")
print(f"[FINAL Model over 120 epochs] Beat F1   : {np.mean(summary_final_beats):.4f} ± {np.std(summary_final_beats):.4f}")
print(f"[FINAL] Downbeat : {np.mean(summary_final_down):.4f} ± {np.std(summary_final_down):.4f}")
print(f"[BEST Model over 120 epochs]  Beat F1   : {np.mean(summary_best_beats):.4f} ± {np.std(summary_best_beats):.4f}")
print(f"[BEST ] Downbeat : {np.mean(summary_best_down):.4f} ± {np.std(summary_best_down):.4f}")


=== 5-Fold Summary (F-measure) (90,215) ===
[FINAL Model over 120 epochs]  Beat F1   : 0.6079 ± 0.0177
[FINAL ] Downbeat : 0.3020 ± 0.0118
[BEST Model over 120 epochs] Beat F1   : 0.6094 ± 0.0177
[BEST] Downbeat : 0.3035 ± 0.0130


In [ ]:
total_params = model.count_params()
param_size_mib = total_params * 4 / 1024**2  # float32 storage size
print(f"Total params: {total_params:,} params  ({param_size_mib:.1f} MiB)")

Total params: 65,962 params  (0.3 MiB)


### 62,962 params >> 0.065 M params >> 0.1 Params

### Note on DBN Parameter Adjustment

We adjusted the parameters of the **Dynamic Bayesian Network (DBN)** used for beat and downbeat tracking.  
For traditional machine learning algorithms, **parameter tuning is essential** for achieving optimal performance.  
However, this also implies that such models often **generalize poorly** across datasets — each dataset tends to have its own "best" set of parameters.

In our case, we modified the DBN beat/downbeat tracker settings:  
- `min_bpm = 90.0` (best observed; we tested [20, 40, 55, 75, 95, 105])  
- `max_bpm = 215.0` (best observed; we tested [150, 180, 200, 215, 240, 275, 300])  

These adjustments improved results on MazurkaBL dataset but also emphasize the **dataset-specific sensitivity** of DBN approaches.

# Backup: Verify our reproduced TCN with GTZAN-mini

Our TCN + DBN implementation achieves performance on GTZAN-mini that is **consistent with the results reported by Böck et al.**, confirming the correctness of our reimplementation. [Colab Notebook of Published TCN w. GTZAN-mini](https://colab.research.google.com/drive/1tuOqNyO9gdMmYJsj33fP_QOfpRsm2tmt?usp=sharing) 

---

**Acknowledgements**  
- **S. Böck & M. E. P. Davies**, *Tempo, Beat, and Downbeat Tutorial* (2020). [eBook](https://tempobeatdownbeat.github.io/tutorial/intro.html)
- **S. Böck & M. E. P. Davies (2020)**, *Deconstruct, Analyse, Reconstruct: How to Improve Tempo, Beat, and Downbeat Estimation*, *Proc. ISMIR*, pp. 574–582. [Paper](https://program.ismir2020.net/static/final_papers/223.pdf)

---

**DBN Tracker Settings for GTZAN-mini**: modify the beat/downbeat tracker settings in `predict_beats_downbeats` before the inference
```python
    def _beat_tracker(beats_act):
        proc = madmom.features.beats.DBNBeatTrackingProcessor(
            min_bpm=55.0, max_bpm=215.0, fps=fps, transition_lambda=100, threshold=0.05
        ...
    def _downbeat_tracker(beats_act, downbeats_act):
        ...
        proc = madmom.features.downbeats.DBNDownBeatTrackingProcessor(
            beats_per_bar=[3, 4], min_bpm=55.0, max_bpm=215.0, fps=fps, transition_lambda=100
        )
```

In [16]:
# ========= DOWNLOAD GTZAN MINI DATASET & RANDOM SPLIT =========
import mirdata
from sklearn.model_selection import train_test_split

gtzan = mirdata.initialize('gtzan_genre', version='mini')
gtzan.download()
tracks = gtzan.load_tracks()

gtzan_train_files, gtzan_test_files = train_test_split(list(tracks.keys()), test_size=0.2, random_state=1234)
print(len(gtzan_train_files), len(gtzan_test_files), gtzan_test_files[-1])

train_ids = [x for x in gtzan_train_files]
valid_ids =  [x for x in gtzan_test_files]
test_ids = valid_ids

80 20 pop.00002


In [ ]:
# ========= GTZAN TRAINING =========
epochs, pad_frames = 120, 2
ckpt_root = Path.cwd() / "gtzan_checkpoint"
pp = PreProcessor()
train = DataSequence({k: tracks[k] for k in train_ids}, pre_processor=pp, pad_frames=pad_frames)
valid = DataSequence({k: tracks[k] for k in valid_ids}, pre_processor=pp, pad_frames=pad_frames)

input_shape = (None,) + train[0][0].shape[-2:]
model = create_model(input_shape)
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss=[build_masked_loss(K.binary_crossentropy)]*2,
              metrics=['binary_accuracy'])
outdir = ckpt_root / f"gtzan_eps{epochs}"
outdir.mkdir(parents=True, exist_ok=True)

cb = [
    keras.callbacks.ModelCheckpoint(str(outdir/"model_best.h5"), save_best_only=True, monitor="val_loss"),
    keras.callbacks.ReduceLROnPlateau(patience=10, factor=0.2, min_lr=1e-7, monitor="val_loss"),
    keras.callbacks.EarlyStopping(patience=80, monitor="val_loss"),
    CSVLogger(str(outdir/"train_log.txt"), append=True, separator="\t")
]
model.fit(train, validation_data=valid, epochs=epochs, callbacks=cb, shuffle=True, verbose=1)
model.save(outdir/"model_final.h5")

In [27]:
# ========= GTZAN EVALUATION =========
beat_ann     = {k: v.beats.times for k, v in tracks.items() if v.beats is not None}
downbeat_ann = {k: v.beats.times[v.beats.positions.astype(int)==1] for k, v in tracks.items() if v.beats is not None}

pp = PreProcessor()
test = DataSequence({k: tracks[k] for k in test_ids}, pre_processor=pp, pad_frames=pad_frames)

test.widen_beat_targets()
test.widen_downbeat_targets()

input_shape = (None,) + test[0][0].shape[-2:]
model = create_model(input_shape)
model.compile(optimizer="adam",
              loss=[build_masked_loss(K.binary_crossentropy)]*2,
              metrics=['binary_accuracy'])

outdir = ckpt_root / f"gtzan_eps{epochs}"
model.load_weights(str(outdir / "model_best.h5"))

detections = predict_beats_downbeats(model, test, fps=pp.fps)
scores = evaluate_beats_and_downbeats(detections, beat_ann, downbeat_ann)

print(f"Beat F1: {scores['beat'].fmeasure:.4f}")
print(f"Downbeat F1: {scores['downbeat'].fmeasure:.4f}")

Predicting: 100%|██████████| 20/20 [00:03<00:00,  6.13it/s]



Beat evaluation
mean for 20 files
  F-measure: 0.821 P-score: 0.799 Cemgil: 0.729 Goto: 0.700 CMLc: 0.613 CMLt: 0.645 AMLc: 0.854 AMLt: 0.892 D: 2.809 Dg: 1.634

Downbeat evaluation
mean for 20 files
  F-measure: 0.305 P-score: 0.252 Cemgil: 0.272 Goto: 0.000 CMLc: 0.000 CMLt: 0.000 AMLc: 0.058 AMLt: 0.066 D: 2.512 Dg: 1.582
Beat F1: 0.8208
Downbeat F1: 0.3052
